In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import pandas as pd
import nltk
from textblob import TextBlob
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mancef\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mancef\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
nltk.download('wordnet')

In [26]:
df = pd.read_csv("data/data.csv")

In [27]:
df.head()

,date,verified,review_title,review_comment,type_of_traveler,start_point,end_point,date_of_flight,overall_rating,recommended,seat_comfort,cabin_staff_service,food_beverages,ground_service,value_for_money,flight_company,airplane_model
0,2020-01-25,True,"""will never fly American again""",Greensboro to Las Vegas via Charlotte. I wil...,Business,Greensboro,Las Vegas via Charlotte,January 2020,1.0,no,1.0,1.0,1.0,1.0,1.0,american-airlines,NaN
1,2020-01-25,False,"""just comfortable enough""",Not Verified | Tokyo to Dallas. I was dreadin...,Solo Leisure,Tokyo,Dallas,January 2020,8.0,yes,3.0,4.0,4.0,4.0,4.0,american-airlines,Boeing 777-200
2,2020-01-24,True,"""never fly with them again""",Providence to Tucson via Chicago. American A...,Solo Leisure,Providence,Tucson via Chicago,January 2020,1.0,no,1.0,1.0,1.0,1.0,1.0,american-airlines,NaN
3,2020-01-24,False,"""Horrible customer service""",Not Verified | Philadelphia to Boston. Horrib...,Business,Philadelphia,Boston,December 2019,1.0,no,1.0,1.0,1.0,1.0,1.0,american-airlines,NaN
4,2020-01-23,True,"""missed my connecting flight""",Houston to Dallas. I had departed from Ameri...,Solo Leisure,Houston,Dallas,January 2020,4.0,no,2.0,3.0,3.0,1.0,1.0,american-airlines,NaN


In [28]:
Reviews = df["review_comment"]
Reviews.head()

0      Greensboro to Las Vegas via Charlotte. I wil...
1    Not Verified |  Tokyo to Dallas. I was dreadin...
2      Providence to Tucson via Chicago. American A...
3    Not Verified |  Philadelphia to Boston. Horrib...
4      Houston to Dallas. I had departed from Ameri...
Name: review_comment, dtype: object

In [29]:
stemmer = SnowballStemmer("english") 

In [12]:
# lemmatize the verbs - bring to the initial form
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))       
    return result

In [30]:
# break reviews to sentences
review_sentences = []
for review in Reviews:
    for x in sent_tokenize(review):
        review_sentences.append(x)

In [31]:
# Clean data
processed_reviews = []
for sentence in review_sentences:
    processed_reviews.append(preprocess(lemmatize_stemming(sentence)))

In [32]:
# Dictionary
dictionary = gensim.corpora.Dictionary(processed_reviews)

In [33]:
# Bag of words
bow_corpus = [dictionary.doc2bow(rev) for rev in processed_reviews]

In [34]:
##### TO DO -  Remove companies' names #####

lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 5,   # choose correct number 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [35]:
# print topics
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")
    

    
# bow_corpus = [dictionary.doc2bow(rev) for rev in processed_reviews]

Topic: 0 
Words: 0.043*"airlin" + 0.029*"servic" + 0.023*"travel" + 0.021*"fli" + 0.021*"custom" + 0.020*"experi" + 0.019*"emir" + 0.016*"time" + 0.014*"cost" + 0.014*"price"


Topic: 1 
Words: 0.110*"flight" + 0.047*"time" + 0.037*"hour" + 0.023*"delay" + 0.020*"arriv" + 0.017*"minut" + 0.014*"leav" + 0.012*"late" + 0.012*"book" + 0.011*"return"


Topic: 2 
Words: 0.082*"seat" + 0.039*"crew" + 0.034*"cabin" + 0.033*"flight" + 0.019*"extra" + 0.017*"good" + 0.013*"comfort" + 0.013*"friend" + 0.010*"aircraft" + 0.010*"plane"


Topic: 3 
Words: 0.054*"board" + 0.046*"check" + 0.027*"staff" + 0.021*"luggag" + 0.018*"airport" + 0.016*"pass" + 0.014*"tell" + 0.013*"print" + 0.013*"passeng" + 0.012*"help"


Topic: 4 
Words: 0.129*"ryanair" + 0.029*"fli" + 0.028*"dubai" + 0.026*"food" + 0.026*"stanst" + 0.025*"return" + 0.023*"drink" + 0.022*"london" + 0.017*"meal" + 0.016*"dublin"




In [36]:
###### Sentiment score #######
sentence_polarity = []
sentence_sensitivity = []
for sentence in review_sentences:
    sentence_polarity.append(TextBlob(sentence).sentiment.polarity)
    sentence_sensitivity.append(TextBlob(sentence).sentiment.subjectivity)
#I have many zeros

In [37]:
sentence_polarity[0:5]

[0.0, -0.2, 0.8, 0.25, 0.1875]

In [38]:
sentence_sensitivity[0:5]

[0.0, 0.45, 0.9, 0.75, 0.6375]